In [1]:
%load_ext autoreload
%autoreload 2

import os
os.chdir('/home/users/pjh/scripts/python_genome_packages/tests/')
from pprint import pprint
import pysam

In [2]:
import pysam
from pprint import pprint
#from importlib import reload

from julib import common
from julib.common import ChromDict, Vcfspec, Interval
from julib.variantplus.breakends import Breakends
from julib.common import cpprint

import julib.annotation

In [3]:
FASTA_PATH = "/home/users/data/01_reference/human_g1k_v37/human_g1k_v37.fasta"
FASTA_PATH_38 = "/home/users/data/01_reference/human_g1k_v38/Homo_sapiens_assembly38.fasta"

#VCF_PATH = "/home/users/pjh/scripts/python_genome_packages/tests/vep_split.vcf.gz"

CHROMDICT = ChromDict(fasta_path = FASTA_PATH)
CHROMDICT_38 = ChromDict(fasta_path = FASTA_PATH_38)
FASTA = pysam.FastaFile(FASTA_PATH)
FASTA_38 = pysam.FastaFile(FASTA_PATH_38)

In [4]:
import time

def timer(func):
    def wrapper(*args, **kwargs):
        ts1 = time.perf_counter()
        result = func(*args, **kwargs)
        ts2 = time.perf_counter()
        print(f'elapsed time: {ts2 - ts1}')
        return result
    return wrapper

In [16]:
vcf_path = '/home/users/pjh/practice/pipeline_test/merge/merged_head.annot.vcf.gz'

In [17]:
from julib.variantplus import vcfplus

In [20]:
vcfp = vcfplus.VcfPlus(vcf_path)

[E::idx_find_and_load] Could not retrieve index file for '/home/users/pjh/practice/pipeline_test/merge/merged_head.annot.vcf.gz'
[E::idx_find_and_load] Could not retrieve index file for '/home/users/pjh/practice/pipeline_test/merge/merged_head.annot.vcf.gz'


In [50]:
bnds = Breakends('21', 39824915, False, '21', 42879675, True, FASTA) # prostate cancer ERG-TMPRSS2
#bnds = Breakends('2', 29_446_413, False, '2', 42_504_396, False, FASTA) # ALK-EML4
vcfspec1 = Vcfspec(bnds.chrom1, bnds.pos1, FASTA.fetch(bnds.chrom1, bnds.pos1-1, bnds.pos1), 'N')
vcfspec2 = Vcfspec(bnds.chrom2, bnds.pos2, FASTA.fetch(bnds.chrom2, bnds.pos2-1, bnds.pos2), 'N')

In [51]:
vep_vr1 = veplib.run_vep_with_vcfspec(vcfspec1, 'hg19')
vep_vr2 = veplib.run_vep_with_vcfspec(vcfspec2, 'hg19')

In [52]:
vep_result1 = ensembl_parser.parse_cmdline_vep(vep_vr1)
vep_result2 = ensembl_parser.parse_cmdline_vep(vep_vr2)

In [53]:
idmap1 = {annotitem['id']: annotitem['refseq_id'] for annotitem in vep_result1['transcript'].values()}
idmap2 = {annotitem['id']: annotitem['refseq_id'] for annotitem in vep_result2['transcript'].values()}

In [54]:
idmap1

{'ENST00000288319': 'NM_182918.3',
 'ENST00000398897': 'NM_001243429.1',
 'ENST00000398905': None,
 'ENST00000398907': None,
 'ENST00000398910': None,
 'ENST00000398911': None,
 'ENST00000398919': 'NM_001243428.1',
 'ENST00000417133': 'NM_001136154.1&NM_001243432.1',
 'ENST00000429727': None,
 'ENST00000442448': 'NM_004449.4',
 'ENST00000453032': 'NM_001136155.1',
 'ENST00000468474': None,
 'ENST00000473107': None,
 'ENST00000481609': None}

In [55]:
idmap2

{'ENST00000332149': 'NM_005656.3',
 'ENST00000398585': 'NM_001135099.1',
 'ENST00000424093': None,
 'ENST00000454499': None,
 'ENST00000455813': None,
 'ENST00000458356': None,
 'ENST00000463138': None,
 'ENST00000497881': None}

In [56]:
#annotdb = annotationdb.AnnotDB('plain', 'hg19', FASTA)
annotdb1 = annotationdb.AnnotDB('bnd1', 'hg19', FASTA)
annotdb2 = annotationdb.AnnotDB('bnd1', 'hg19', FASTA)

In [57]:
#annotdb.update_features_plain(vcfspec, julib.annotation.TABIXFILES_REPEATS['hg19'])
annotdb1.update_features_bnd(
    bnds.chrom1, bnds.pos1, bnds.pos1_endis5, 
    julib.annotation.TABIXFILES_REPEATS['hg19'], 
    julib.annotation.TABIXFILES_GENESET['hg19'],
    julib.annotation.TABIXFILES_REGULATORY['hg19'],
)

annotdb2.update_features_bnd(
    bnds.chrom2, bnds.pos2, bnds.pos2_endis5, 
    julib.annotation.TABIXFILES_REPEATS['hg19'], 
    julib.annotation.TABIXFILES_GENESET['hg19'],
    julib.annotation.TABIXFILES_REGULATORY['hg19'],
)

AttributeError: module 'julib.annotation.rnalib' has no attribute 'set_next_codon_frame0'

In [ ]:
for ID, annotitem in annotdb1.transcript.items():
    print(ID, annotitem['transcript_name'], idmap1[ID], annotitem['next_codon_frame0'], sep='\t')

In [ ]:
for ID, annotitem in annotdb2.transcript.items():
    print(ID, annotitem['transcript_name'], idmap2[ID], annotitem['next_codon_frame0'], sep='\t')

In [58]:
next_codon_frame0_pos1 = rnalib.get_next_codon_frame0(
    [x for x,y in vep_result1['transcript'].items() if y['distance'] is None], 
    bnds.chrom1, bnds.pos1, bnds.pos1_endis5, julib.annotation.TABIXFILES_GENESET['hg19'])

next_codon_frame0_pos2 = rnalib.get_next_codon_frame0(
    [x for x,y in vep_result2['transcript'].items() if y['distance'] is None], 
    bnds.chrom2, bnds.pos2, bnds.pos2_endis5, julib.annotation.TABIXFILES_GENESET['hg19'])

In [59]:
print(next_codon_frame0_pos1)
print(next_codon_frame0_pos2)

{'ENST00000288319': 0, 'ENST00000398897': None, 'ENST00000398905': 0, 'ENST00000398907': 0, 'ENST00000398910': 0, 'ENST00000398911': 0, 'ENST00000398919': 0, 'ENST00000417133': 0, 'ENST00000429727': 0, 'ENST00000442448': 0, 'ENST00000453032': None, 'ENST00000468474': None, 'ENST00000481609': None}
{'ENST00000332149': None, 'ENST00000398585': 0, 'ENST00000424093': None, 'ENST00000454499': None, 'ENST00000458356': None, 'ENST00000463138': None, 'ENST00000497881': None}


In [192]:
#from julib.annotation import ensembl_rest
#from julib.annotation import ensembl_parser
from julib import hgvs
#from julib.annotation import veplib
from julib.annotation import annotationdb
from julib.variantplus import breakends

In [196]:
hgvsg = 'chr12:g.25245350C>A' # KRAS hotspot
vcfspec = hgvs.hgvsg_to_vcfspec(hgvsg, FASTA_38)

In [203]:
vcfspec

Vcfspec(chrom='chr12', pos=25245350, ref='C', alt='A')

In [194]:
hgvsg = 'chr12:g.25245350C>A' # KRAS hotspot
#hgvsg = 'chr17:g.7514616_7514617del'
#hgvsg = 'chr17:g.7514615_7514616insTTT'
#hgvsg = 'chr17:g.7514616A>T'

vcfspec = hgvs.hgvsg_to_vcfspec(hgvsg, FASTA_38)

In [26]:
interval = common.Interval('chr17', 7_511_516, 7_512_481)

In [284]:
#bnds = breakends.Breakends('2', 29_446_413, False, '2', 42_504_396, False, FASTA)
#bnds = breakends.Breakends('2', 29_446_413, False, '2', 42_504_396, False, FASTA)
bnds = breakends.Breakends('2', 29_446_413, False, '2', 42_504_396, False, FASTA)
#bnds = breakends.Breakends('21', 39824915, False, '21', 42879675, True, FASTA) # prostate cancer ERG-TMPRSS2

In [8]:
from julib.annotation import ensembl_rest

In [10]:
result = ensembl_rest.vep(
    refver = 'hg19', vcfspec = common.Vcfspec('2', 29444010, 'C', 'N'),
)

In [11]:
result

[{'seq_region_name': '2',
  'start': 29444010,
  'id': '2:g.29444010C>N',
  'allele_string': 'C/N',
  'end': 29444010,
  'most_severe_consequence': 'coding_sequence_variant',
  'transcript_consequences': [{'gene_id': 'ENSG00000171094',
    'consequence_terms': ['intron_variant'],
    'variant_allele': 'N',
    'protein_id': 'ENSP00000373700',
    'gene_symbol_source': 'HGNC',
    'gene_symbol': 'ALK',
    'hgnc_id': 427,
    'impact': 'MODIFIER',
    'transcript_id': 'ENST00000389048',
    'strand': -1,
    'biotype': 'protein_coding',
    'intron': '22/28',
    'canonical': 1,
    'ccds': 'CCDS33172.1'},
   {'gene_symbol_source': 'HGNC',
    'gene_symbol': 'ALK',
    'protein_id': 'ENSP00000414027',
    'hgnc_id': 427,
    'impact': 'MODIFIER',
    'variant_allele': 'N',
    'consequence_terms': ['intron_variant'],
    'gene_id': 'ENSG00000171094',
    'intron': '2/3',
    'biotype': 'protein_coding',
    'strand': -1,
    'transcript_id': 'ENST00000431873'},
   {'cdna_end': 303,
    

In [227]:
bnds

<Breakends> ]2:29446413]  ]2:42504396]

In [228]:
#annotdb = annotationdb.AnnotDB(annotdb_type = 'plain', refver = 'hg38', fasta = FASTA_38)

annotdb_bnd1 = annotationdb.AnnotDB(annotdb_type = 'bnd1', refver = 'hg19', fasta = FASTA)
annotdb_bnd2 = annotationdb.AnnotDB(annotdb_type = 'bnd2', refver = 'hg19', fasta = FASTA)

In [229]:
annotdb_bnd1.update_features_bnd(bnds.chrom1, bnds.pos1, bnds.pos1_endis5)
annotdb_bnd2.update_features_bnd(bnds.chrom2, bnds.pos2, bnds.pos2_endis5)

In [232]:
annotdb_bnd1.transcript

{'ENST00000389048': {'aa_change': None,
                     'biotype': 'protein_coding',
                     'cadd_phred': None,
                     'cadd_raw': None,
                     'ccds_id': 'CCDS33172.1',
                     'chrom': '2',
                     'codon_change': None,
                     'consequence_flags': {'is_3pUTR_involved': False,
                                           'is_5pUTR_involved': False,
                                           'is_SV_consequence': False,
                                           'is_frameshift': False,
                                           'is_missense': False,
                                           'is_not_protein_altering': True,
                                           'is_protein_altering': False,
                                           'is_splice_acceptor_involved': False,
                                           'is_splice_donor_involved': False,
                                           'is_splic

In [277]:
bnds = breakends.Breakends('2', 29_446_413, False, '2', 29_447_413, False, FASTA)

In [278]:
interval = common.Interval(bnds.chrom1, bnds.pos1, bnds.pos2)

In [279]:
annotdb_interval = annotationdb.AnnotDB(annotdb_type = 'plain', refver = 'hg19', fasta = FASTA)

In [280]:
annotdb_interval.update_features_interval(interval)

2:g.29446413_29447413del


In [283]:
[feature for feature in annotdb_interval.repeat.values() if feature['is_enclosed']]

[{'chrom': '2',
  'distance': None,
  'end0': 29447062,
  'end1': 29447062,
  'feature_type': 'repeat',
  'feature_type_flags': {'is_motif': False,
                         'is_regulatory': False,
                         'is_repeat': True,
                         'is_transcript': False},
  'id': '2_29446648_29447062_LTR16B2',
  'is_enclosed': True,
  'is_forward': False,
  'repeat_type': 'LTR16B2',
  'start0': 29446648,
  'start1': 29446649},
 {'chrom': '2',
  'distance': None,
  'end0': 29447359,
  'end1': 29447359,
  'feature_type': 'repeat',
  'feature_type_flags': {'is_motif': False,
                         'is_regulatory': False,
                         'is_repeat': True,
                         'is_transcript': False},
  'id': '2_29447094_29447359_MIRc',
  'is_enclosed': True,
  'is_forward': True,
  'repeat_type': 'MIRc',
  'start0': 29447094,
  'start1': 29447095}]

In [269]:
from julib.svlib import structvars
sv_del = structvars.Deletion(bnds.chrom1, bnds.pos1, bnds.pos2, fasta = FASTA)

In [271]:
result = ensembl_rest.vep('hg19', hgvsg = sv_del.get_hgvsg())

In [163]:
def myfun(pos1_endis5, pos2_endis5, annotdb_bnd1, annotdb_bnd2):
    from itertools import product
    
    if pos1_endis5 != pos2_endis5:
        def strand_checker(feature_bnd1, feature_bnd2):
            return feature_bnd1['is_forward'] == feature_bnd2['is_forward']
    else:
        def strand_checker(feature_bnd1, feature_bnd2):
            return feature_bnd1['is_forward'] != feature_bnd2['is_forward']
        
    if pos1_endis5:
        def codon_frame_checker(feature_bnd1, feature_bnd2):
            bnd1_precedes = not feature_bnd1['is_forward']
            codon_frame0_bnd1 = feature_bnd1['codon_frame0']
            codon_frame0_bnd2 = feature_bnd2['codon_frame0']
            if bnd1_precedes:
                return ((codon_frame0_bnd1 + 1)%3) == (codon_frame0_bnd2%3)
            else:
                return ((codon_frame0_bnd2 + 1)%3) == (codon_frame0_bnd1%3)
    else:
        def codon_frame_checker(feature_bnd1, feature_bnd2):
            bnd1_precedes = feature_bnd1['is_forward']
            codon_frame0_bnd1 = feature_bnd1['codon_frame0']
            codon_frame0_bnd2 = feature_bnd2['codon_frame0']
            if bnd1_precedes:
                return ((codon_frame0_bnd1 + 1)%3) == (codon_frame0_bnd2%3)
            else:
                return ((codon_frame0_bnd2 + 1)%3) == (codon_frame0_bnd1%3) 
        
    for feature_bnd1, feature_bnd2 in product(annotdb_bnd1.transcript.values(), annotdb_bnd2.transcript.values()):
        if strand_checker(feature_bnd1, feature_bnd2):
            if 'codon_frame0' in feature_bnd1 and 'codon_frame0' in feature_bnd2:
                if codon_frame_checker(feature_bnd1, feature_bnd2):
                    print(feature_bnd1['id'], feature_bnd2['id'])

In [164]:
myfun(bnds.pos1_endis5, bnds.pos2_endis5, annotdb_bnd1, annotdb_bnd2)

In [139]:
len(annotdb_bnd1.transcript)

14

In [140]:
len(annotdb_bnd2.transcript)

7

In [141]:
annotdb_bnd2.transcript_canonical

{'ENST00000398585': {'aa_change': None,
                     'biotype': 'protein_coding',
                     'cadd_phred': None,
                     'cadd_raw': None,
                     'ccds_id': 'CCDS54486.1',
                     'chrom': '21',
                     'codon_change': None,
                     'consequence_flags': {'is_3pUTR_involved': False,
                                           'is_5pUTR_involved': False,
                                           'is_SV_consequence': False,
                                           'is_frameshift': False,
                                           'is_missense': False,
                                           'is_not_protein_altering': True,
                                           'is_protein_altering': False,
                                           'is_splice_acceptor_involved': False,
                                           'is_splice_donor_involved': False,
                                           'is_spli

In [90]:
from julib.annotation import ensembl_rest

In [106]:
%%timeit
result = ensembl_rest.vep(
    refver = 'hg19', 
    vcfspec = common.Vcfspec('2', 29_446_394, 'A', 'N'), 
    distance = 0,
    with_CADD = False,
    with_Phenotypes = False,
    with_canonical = False,
    with_mane = False,
    with_miRNA = False,
    with_numbers = False,
    with_protein = True,
    with_ccds = False,
    with_hgvs = False,
)

1.03 s ± 326 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [107]:
%%timeit
result = ensembl_rest.vep(
    refver = 'hg19', 
    vcfspec = common.Vcfspec('2', 29_446_394, 'A', 'N'), 
    distance = 0,
)

897 ms ± 51.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [101]:
result

[{'start': 29446394,
  'assembly_name': 'GRCh37',
  'input': '2:g.29446394A>N',
  'transcript_consequences': [{'cdna_end': 4080,
    'gene_symbol_source': 'HGNC',
    'gene_id': 'ENSG00000171094',
    'codons': 'gTg/gNg',
    'protein_start': 1058,
    'cdna_start': 4080,
    'hgnc_id': 427,
    'consequence_terms': ['splice_region_variant', 'coding_sequence_variant'],
    'variant_allele': 'N',
    'cds_end': 3173,
    'biotype': 'protein_coding',
    'impact': 'LOW',
    'protein_end': 1058,
    'protein_id': 'ENSP00000373700',
    'transcript_id': 'ENST00000389048',
    'strand': -1,
    'gene_symbol': 'ALK',
    'cds_start': 3173},
   {'gene_id': 'ENSG00000171094',
    'gene_symbol_source': 'HGNC',
    'protein_id': 'ENSP00000414027',
    'strand': -1,
    'variant_allele': 'N',
    'consequence_terms': ['intron_variant'],
    'hgnc_id': 427,
    'transcript_id': 'ENST00000431873',
    'biotype': 'protein_coding',
    'gene_symbol': 'ALK',
    'impact': 'MODIFIER'}],
  'end': 29446

In [155]:
result1 = ensembl_rest.vep(
    vcfspec = vcfspec, 
    #hgvsg = hgvsg,
    refver = 'hg38', 
    distance = 100,
)

In [156]:
parsed_result = ensembl_parser.parse_rest_vep(result1)

In [42]:
vep_vr = veplib.run_vep_with_vcfspec(vcfspec, 'hg38', 100)

In [44]:
parsed_result_cmdvep = ensembl_parser.parse_cmdline_vep(vep_vr)

In [8]:
from julib.variantplus import vcfplus

In [88]:
#vcf_path = '/home/users/pjh/practice/pipeline_test/vep_julib/input.processed.vep.vcf.gz'
#vcf_path = '/home/users/pjh/practice/pipeline_test/vep_julib/manta_subsample.processed.vep.vcf.gz'
vcf_path = '/home/users/pjh/practice/pipeline_test/vep_julib/sv_testinput.processed.vep.vcf.gz'

In [89]:
vcfp = vcfplus.VcfPlus(vcf_path, FASTA_38, CHROMDICT_38)

[E::idx_find_and_load] Could not retrieve index file for '/home/users/pjh/practice/pipeline_test/vep_julib/sv_testinput.processed.vep.vcf.gz'


In [92]:
vp = vcfp.vplist[0]
vp.annotdb_interval.update_features(hg19 = False)

In [84]:
from julib.variantplus.variantplus import VariantPlus

In [85]:
delly_vcf_path = '/home/users/pjh/practice/pipeline_test/example_SV_vcfs_by_caller/delly_wrapper_test_v1.2_211122.vcf.gz'
#delly_vcf_path = '/home/users/team_projects/Lung_Cancer_Panel_data/03_Data_from_YTKim/04_delly/14.tumor/14.tumor.delly.DEL.bcf'
standard_vcf_path = '/home/users/pjh/tmp/delly_standard.vcf.gz'

In [98]:
vcf = pysam.VariantFile(standard_vcf_path)
fetcher = vcf.fetch()

[E::idx_find_and_load] Could not retrieve index file for '/home/users/pjh/tmp/delly_standard.vcf.gz'


In [100]:
while True:
    vp = VariantPlus(next(fetcher))
    if vp.bnds.svtype != 'TRA':
        break
    

In [102]:
print(vp.vr)

1	738610	1_738610_1_742021_1	A	A[1:742021[	0	PASS	PRECISE;SVTYPE=DEL;SVMETHOD=EMBL.DELLYv0.7.3;CHR2=1;INSLEN=0;PE=11;MAPQ=22;CT=3to5;CIPOS=-251,251;CIEND=-251,251;SR=28;SRQ=1;CONSENSUS=GATGGAAAAAATGATTTTACATAAAACATTAGTTGAAAGATACAGGCTGGGCATGGTGGCTCACACCTGTAATCCCAACACTTTGGGAGGCCGAGGCAGGTGGATCACTTGAGGTCAGGAGTTCAGTACTGTAAACACTGCTGCTATTCTCTTAGATGTTTTTCTTCCATGATATACCCTTATCCTGTATTGGTTTGTATAGATTAGCACATTAGGTTTCAGAAGTCATTTTAAAC;CE=1.96531;callerInfo=Delly@LY17_C;MATEID=1_738610_1_742021_2	GT:GL:GQ:FT:RCL:RC:RCR:CN:DR:DV:RR:RV	0/1:-19.2679,0,-71.1927:10000:PASS:906:627:568:1:23:11:23:12	0/1:-9.83583,0,-50.5986:98:PASS:421:393:302:1:21:8:17:7



In [103]:
vp.bnds

{'chrom1': '1',
 'chrom2': '1',
 'inserted_seq': [],
 'pos1': 738610,
 'pos1_endis5': False,
 'pos2': 742021,
 'pos2_endis5': True,
 'svtype': 'DEL'}

In [104]:
vp.annotdb_interval.update_features(hg19 = True)